In [11]:
import requests
from bs4 import BeautifulSoup
import pandas

In [12]:
# constants here
CATEGORIES = ['activewear', 'tops', 'bottoms', 'denim', 'dresses', 'knitwear', 'outerwear', 'suits', 'basics', 'sleepwear', 'swimwear', 'maternity', 'plus-size', 'shoes', 'bags', 'accessories']

CATEGORY_ROOT_URL = 'https://directory.goodonyou.eco/categories/'

BRAND_ROOT_URL = 'https://directory.goodonyou.eco/brand/'

In [13]:
# if a store has already been scraped, just add the category to this map and do not "rescrape" to save time
# we assume there is only one unique page per brand

STORE_CATEGORY_MAP = {}

In [14]:
# brand page specifics
# maps field name to ['div type', 'class name'], #example text
BRAND_SCRAPING_MAP = {
    'name': ['h1', 'sc-kjoXOD hxRXn StyledHeading-sc-1rdh4aw-0 iadRcX'], # 'Article 22'
    'rating': ['span', 'StyledText-sc-1sadyjn-0 bBUTWf'], # 'Rated: Good'
    'price': ['span', 'StyledText-sc-1sadyjn-0 bBUTWf'], # 'Price : $$'
    'location': ['span', 'StyledText-sc-1sadyjn-0 bBUTWf'], # 'Location: Australia'
    'planet': ['span', 'StyledText-sc-1sadyjn-0 bVvIwM'], # '5 out of 5'
    'people': ['span', 'StyledText-sc-1sadyjn-0 bVvIwM'], # '4 out of 5', 'Not applicable'
    'animals': ['span', 'StyledText-sc-1sadyjn-0 bVvIwM'], # same as above
    'description': ['div', 'sc-kgAjT kqcGSN'], # all text comes in chunks of <p>'s
}